#### Da wir ein sehr schlankes Model zur Sentimentanalyse brauchten, damit es auf dem Free-Tier Server mit stark begrenzten Ressourcen, der unser Backend hosten soll läuft, wurden verschiedenste Modelle getestet. Größere Sentiment-Modelle, die relativ gut performt haben, waren zu groß um auf dem Server zu laufen. Allerdings hatten alle getesteten Modelle das prblem, das sie lediglich auf positiv, neutral und negativ trainiert waren. Das hatte besonders bei Beleidigiungen, die die Modelle möglicherweise beim Training nicht gesehen haben ein Problem, da diese oft als freundlich oder neutral gelabelt wurden. Aus diesem Grund entschlossen wir uns das größte Bert Modell zu nehmen, welches wir auf dem Free-Tier Server hosten konnten (Tinybert) und dieses auf einem Datensatz freundlicher, neutraler und unfreundlicher Sätze finezutunen.

#### Imports

In [1]:
import os
import csv
import time
import json
import openai

#### Umgebungsvariablen setzen

In [2]:
# OpenAI Api Schlüssel
openai.api_key = os.getenv("OPENAI_API_KEY")

# Dateiname für die Ausgabe
OUTPUT_CSV = "sentences.csv"

# Training-Parameter
LABELS = ["sehr freundlich", "neutral", "unfreundlich"]
NUM_PER_LABEL = 3000 
BATCH_SIZE = 25
DELAY_BETWEEN_CALLS = 0.5

#### CSV-Datei mit den Spaltenüberschriften "label" und "sentence" initialisieren

In [3]:
if not os.path.isfile(OUTPUT_CSV):
    with open(OUTPUT_CSV, "w", newline='', encoding="utf-8") as f:
        csv.writer(f).writerow(["label", "sentence"])

#### Funktion zum Parsen von JSON-Antworten in einzelne Sätze

In [4]:
def parse_sentences(text):
    """
    Extrahiert Sätze aus einem JSON-formatierten String. 
    Falls kein gültiges JSON vorliegt, wird zeilenweise geparst.

    Args: text (str): Ein JSON-String mit dem Schlüssel "sentences" oder roher Text, in dem jede Zeile einen Satz darstellt.

    Returns: List[str]: Eine Liste der extrahierten Sätze.
    """
    try:
        data = json.loads(text)
        return data.get("sentences", [])
    except json.JSONDecodeError:
        # Fallback: Zeilenweise parsen
        lines = [l.strip() for l in text.splitlines() if l.strip()]
        return lines

#### Funktion zum Schreiben der Sätze in die CSV

In [5]:
def append_to_csv(path, label, sentences):
    """
    Hängt eine Liste von Sätzen mit zugehörigem Label an eine bestehende CSV-Datei an.

    Args: path (str): Der Dateipfad zur CSV-Datei.
          label (str): Das Label, das jeder Satz erhalten soll.
          sentences (List[str]): Eine Liste von Sätzen, die angehängt werden sollen.

    Returns: None
    """
    with open(path, "a", newline='', encoding="utf-8") as f:
        writer = csv.writer(f)
        for s in sentences:
            writer.writerow([label, s])
        f.flush()


#### ChatGPT Prompt zum erstellen des Datensatzes

In [6]:
system_message = {
    "role": "system",
    "content": (
        "Du bist ein Datengenerator, der Chatnachrichten eines Minecraft-Spielers an einen Villager-NPC erzeugt."
        " Jede Nachricht soll dem Stil eines echten Spielers entsprechen (Anfragen, Kommentare, Beleidigungen etc.),"
        " ohne Metadaten oder Erklärungen. Halte dabei deine Sätze relativ allgemein und formuliere sie nicht zu Minecraft spezifisch. Antworte immer im JSON-Format wie:\n"
        "{\n  \"sentences\": [\"...\", \"...\"]\n}\n"
        "Beispiele pro Label:\n"
        "# sehr freundlich:\n"
        "[\"Hallo netter Villager! Welche Trades kannst du mir anbieten?\", \"Danke fürs Handeln, du bist großartig!\"]\n"
        "# neutral:\n"
        "[\"Welchen Beruf hast du?\", \"Wie viel Uhr ist es aktuell?\"]\n"
        "# unfreundlich:\n"
        "[\"Hey, du Idiot, gib mir bessere Trades!\", \"Du kleiner Hurensohn, sag mir endlich wie viel Uhr es ist!\"]"
    )
}

#### Gernerieren und Speichern des Datensatzes

In [7]:
# Zähler für jedes Label initialisieren
counts = {lbl: 0 for lbl in LABELS}

# Solange noch nicht genug Sätze pro Label existieren:
while any(counts[l] < NUM_PER_LABEL for l in LABELS):
    for label in LABELS:
        rem = NUM_PER_LABEL - counts[label]
        if rem <= 0:
            continue

        batch = min(BATCH_SIZE, rem)
        # User-Prompt anpassen
        user_content = f"Erzeuge bitte {batch} kurze Chatnachrichten, die '{label}' sind."
        if label == "unfreundlich":
            user_content += (
                " Verwende dabei auch harte, deutsche Beleidigungen "
                "(z.B. Bastard, Hurensohn, Wichser, Arschloch, Idiot usw.)."
            )
        user_message = {"role": "user", "content": user_content}

        # API-Aufruf und Parsing
        try:
            resp = openai.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[system_message, user_message],
                temperature=0.8,
                max_tokens=batch * 25
            )
            sents = parse_sentences(resp.choices[0].message.content)[:batch]
            append_to_csv(OUTPUT_CSV, label, sents)
            counts[label] += len(sents)
            print(f"{label}: +{len(sents)} (insgesamt {counts[label]}/{NUM_PER_LABEL})")
        except Exception as e:
            print(f"Fehler bei Label '{label}': {e}")
            time.sleep(5)
            continue

        time.sleep(DELAY_BETWEEN_CALLS)

print(f"Fertig! Alle Labels haben jeweils {NUM_PER_LABEL} Sätze. Datei: {OUTPUT_CSV}")

sehr freundlich: +25 (insgesamt 25/3000)


KeyboardInterrupt: 

#### Nach einer ersten Durchsicht des Datensatzen viel auf, dass ChatGPT kaum in der Lage ist Beleidigungen in den Datensatz einzubauen. Aus diesem Grund entschlossen wir uns dazu den Datensatz zu erweitern und dieses Mal Platzhalter für die Beleidigungen einfügen zu lassen, die wir später gegen Beleidigungen ersetzen können.

#### Überarbeiteter ChatGPT Prompt

In [8]:
new_system_message = {
    "role": "system",
    "content": (
        "Du bist ein Datengenerator, der Chatnachrichten eines Minecraft-Spielers an einen Villager-NPC erzeugt. "
        "Jede Nachricht soll dem Stil eines echten Spielers entsprechen (Anfragen, Kommentare, Beleidigungen etc.), "
        "ohne Metadaten oder Erklärungen. Halte dabei deine Sätze relativ allgemein und formuliere sie nicht zu Minecraft-spezifisch. "
        "Antworte immer exakt im JSON-Format:\n"
        "{\n  \"sentences\": [\"...\", \"...\"]\n}\n\n"
        "–––\n"
        "Für unfreundliche Nachrichten: Verwende statt direkter Beleidigungen *genau einen* der folgenden 13 Platzhalter:\n"
        "{P1}, {P2}, {P3}, {P4}, {P5}, {P6}, {P7}, {P8}, {P9}, {P10}, {P11}, {P12}, {P13}\n"
        "Diese Platzhalter werden später durch echte Schimpfwörter ersetzt.\n"
        "–––\n"
        "Beispiele pro Label:\n"
        "# sehr freundlich:\n"
        "[\"Hallo netter Villager! Welche Trades kannst du mir anbieten?\", \"Danke fürs Handeln, du bist großartig!\"]\n"
        "# neutral:\n"
        "[\"Welchen Beruf hast du?\", \"Wie viel Uhr ist es aktuell?\"]\n"
        "# unfreundlich:\n"
        "[\"Hey {P5}, gib mir bessere Trades!\", \"Du {P11}, sag mir endlich wie viel Uhr es ist!\"]"
    )
}

#### Datensatz mit Platzhaltern für Beleidingungen bei unfreundlichen Sätzen erweitern

In [9]:
NEW_NUM_PER_LABEL = 2000 

# Zähler für jedes Label initialisieren
counts = {lbl: 0 for lbl in LABELS}

# Solange noch nicht genug Sätze pro Label existieren:
while any(counts[lbl] < NEW_NUM_PER_LABEL for lbl in LABELS):
    for label in LABELS:
        remaining = NEW_NUM_PER_LABEL - counts[label]
        if remaining <= 0:
            continue

        # Erzeuge bis zu BATCH_SIZE pro Aufruf
        batch = min(BATCH_SIZE, remaining)
        # User-Message mit Batch und Label
        user_content = f"Erzeuge bitte {batch} kurze Chatnachrichten, die '{label}' sind."
        if label == "unfreundlich":
            user_content += (
                " Verwende dabei *pro Satz* genau einen der folgenden Platzhalter: "
                "{P1}, {P2}, {P3}, {P4}, {P5}, {P6}, {P7}, {P8}, {P9}, {P10}, {P11}, {P12}, {P13}."
            )
        user_message = {"role": "user", "content": user_content}

        try:
            resp = openai.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[new_system_message, user_message],
                temperature=0.8,
                max_tokens=batch * 25
            )
            sent_list = parse_sentences(resp.choices[0].message.content)
            sent_list = sent_list[:batch]
            append_to_csv(OUTPUT_CSV, label, sent_list)
            counts[label] += len(sent_list)
            print(f"{label}: +{len(sent_list)} (insgesamt {counts[label]}/{NEW_NUM_PER_LABEL})")
        except Exception as e:
            print(f"Fehler bei Label '{label}': {e}")
            time.sleep(5)
            continue

        time.sleep(DELAY_BETWEEN_CALLS)

print(f"Fertig! Alle Labels haben jeweils {NEW_NUM_PER_LABEL} Sätze. Datei: {OUTPUT_CSV}")

sehr freundlich: +25 (insgesamt 25/2000)
neutral: +25 (insgesamt 25/2000)
unfreundlich: +25 (insgesamt 25/2000)
sehr freundlich: +24 (insgesamt 49/2000)


KeyboardInterrupt: 